# 1. Data Processing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 1.1 Import and inspect the data

In [ ]:
# Import the data to have an overview
df = pd.read_csv('Hospital_Inpatient_Discharges__SPARCS_De-Identified___Cost_Transparency__Beginning_2009_20250426.csv')
df.head()

,Year,Facility Id,Facility Name,APR DRG Code,APR Severity of Illness Code,APR DRG Description,APR Severity of Illness Description,APR Medical Surgical Code,APR Medical Surgical Description,Discharges,Mean Charge,Median Charge,Mean Cost,Median Cost
0,2016,4,Albany Memorial Hospital,194,1,Heart Failure,Minor,M,Medical,2,"8,375.41","8,375.41","3,585.05","3,585.05"
1,2016,4,Albany Memorial Hospital,194,2,Heart Failure,Moderate,M,Medical,40,"14,029.82","12,176.95","6,182.67","5,253.15"
2,2016,4,Albany Memorial Hospital,194,3,Heart Failure,Major,M,Medical,70,"23,921.77","20,229.81","11,149.49","9,068.10"
3,2016,4,Albany Memorial Hospital,194,4,Heart Failure,Extreme,M,Medical,12,"51,260.45","35,210.82","26,081.70","15,230.62"
4,2016,4,Albany Memorial Hospital,196,4,Cardiac Arrest,Extreme,M,Medical,1,"25,357.84","25,357.84","7,791.75","7,791.75"


In [ ]:
# Understand the size of data from the number of rows and columnss
df.shape

(1192827, 14)

### 1.2 Handle missing values

In [ ]:
# Check missing value
df.isna().sum()

,0
Year,0
Facility Id,0
Facility Name,0
APR DRG Code,0
APR Severity of Illness Code,0
APR DRG Description,0
APR Severity of Illness Description,210
APR Medical Surgical Code,479
APR Medical Surgical Description,0
Discharges,0


In [ ]:
# Fill the missing value according to the information from data dictionary

# The APR SOI Description: Not Applicable (0), Minor (1), Moderate (2), Major(3) , Extreme (4).
df['APR Severity of Illness Description'].fillna('Not Applicable', inplace=True)

# The APR-DRG code indicating ‘M’ (Medical), ‘P’ (Surgical) or ‘NA’ (Other).
df['APR Medical Surgical Code'].fillna('NA', inplace=True)

<ipython-input-74-0c9a96a1629f>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['APR Severity of Illness Description'].fillna('Not Applicable', inplace=True)
<ipython-input-74-0c9a96a1629f>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method

In [ ]:
# Check if there are still any missing value
df.isna().sum()

,0
Year,0
Facility Id,0
Facility Name,0
APR DRG Code,0
APR Severity of Illness Code,0
APR DRG Description,0
APR Severity of Illness Description,0
APR Medical Surgical Code,0
APR Medical Surgical Description,0
Discharges,0


In [ ]:
# Check if the illness description matches the illness code correctly
df.groupby(['APR Severity of Illness Description', 'APR Severity of Illness Code']).size()

,,0
APR Severity of Illness Description,APR Severity of Illness Code,
Extreme,4,196347
Major,3,325032
Minor,1,309809
Moderate,2,361079
Not Applicable,0,560


In [ ]:
# Check if the the surgical description matches surgical code correctly
df.groupby(['APR Medical Surgical Code', 'APR Medical Surgical Description']).size()

,,0
APR Medical Surgical Code,APR Medical Surgical Description,
M,Medical,805932
NA,Other,479
P,Surgical,386335
U,Other,81


In [ ]:
df['APR Medical Surgical Code'] = df['APR Medical Surgical Code'].replace('U', 'NA')
df.groupby(['APR Medical Surgical Code', 'APR Medical Surgical Description']).size()

,,0
APR Medical Surgical Code,APR Medical Surgical Description,
M,Medical,805932
NA,Other,560
P,Surgical,386335


### 1.3 Handle duplicates

In [ ]:
# Identify duplicates based on selected columns
check_cols = ['Year', 'Facility Id', 'APR DRG Code', 'APR Severity of Illness Code', 'APR Medical Surgical Code']

duplicated_mask = df[df.duplicated(subset=check_cols, keep=False)]
duplicated_mask

,Year,Facility Id,Facility Name,APR DRG Code,APR Severity of Illness Code,APR DRG Description,APR Severity of Illness Description,APR Medical Surgical Code,APR Medical Surgical Description,Discharges,Mean Charge,Median Charge,Mean Cost,Median Cost
1151446,2021,1165,Jacobi Medical Center,956,0,UNGROUPABLE,Not Applicable,NA,Other,2,"29,513.62","29,513.62","17,517.45","17,517.45"
1152370,2021,1165,Jacobi Medical Center,956,0,UNGROUPABLE,Not Applicable,NA,Other,1,"8,194.53","8,194.53","4,863.76","4,863.76"
1152371,2021,1168,Montefiore Medical Center-Wakefield Hospital,956,0,UNGROUPABLE,Not Applicable,NA,Other,6,"125,099.85","62,031.30","24,258.29","15,984.02"
1153105,2021,1168,Montefiore Medical Center-Wakefield Hospital,956,0,UNGROUPABLE,Not Applicable,NA,Other,1,"20,859.06","20,859.06","2,765.50","2,765.50"
1153106,2021,1169,Montefiore Medical Center - Henry & Lucy Moses...,956,0,UNGROUPABLE,Not Applicable,NA,Other,4,"387,953.17","378,035.31","75,840.60","70,953.70"
1154162,2021,1169,Montefiore Medical Center - Henry & Lucy Moses...,956,0,UNGROUPABLE,Not Applicable,NA,Other,4,"28,445.81","28,439.12","6,825.67","6,752.87"
1189427,2021,3058,Montefiore Med Center - Jack D Weiler Hosp of ...,956,0,UNGROUPABLE,Not Applicable,NA,Other,3,"201,112.50","78,737.24","34,993.56","16,831.43"
1190318,2021,3058,Montefiore Med Center - Jack D Weiler Hosp of ...,956,0,UNGROUPABLE,Not Applicable,NA,Other,1,"8,300.00","8,300.00","1,197.23","1,197.23"


In [ ]:
# Check the data types of each column before aggregation
df.dtypes

,0
Year,int64
Facility Id,int64
Facility Name,object
APR DRG Code,int64
APR Severity of Illness Code,int64
APR DRG Description,object
APR Severity of Illness Description,object
APR Medical Surgical Code,object
APR Medical Surgical Description,object
Discharges,object


In [ ]:
# Remove commas first, then convert to numeric
for col in ['Discharges', 'Mean Charge', 'Mean Cost', 'Median Charge', 'Median Cost']:
    df[col] = df[col].str.replace(',', '').astype(float)

df['Discharges'] = df['Discharges'].astype(int)
df['Mean Charge'] = df['Mean Charge'].astype(float)
df['Mean Cost'] = df['Mean Cost'].astype(float)
df['Median Charge'] = df['Median Charge'].astype(float)
df['Median Cost'] = df['Median Cost'].astype(float)

In [ ]:
# Columns to define a "duplicate group"
check_cols = ['Year', 'Facility Id', 'Facility Name', 'APR DRG Code', 'APR DRG Description',
             'APR Severity of Illness Code', 'APR Severity of Illness Description',
             'APR Medical Surgical Code', 'APR Medical Surgical Description']

# Step 1: Clean numeric columns: remove commas and convert to float
for col in ['Discharges', 'Mean Charge', 'Mean Cost', 'Median Charge', 'Median Cost']:
    df[col] = df[col].astype(str).str.replace(',', '').astype(float)

# Step 2: Split into duplicated and non-duplicated parts
duplicated_mask = df.duplicated(subset=check_cols, keep=False)
duplicated_rows = df[duplicated_mask].copy()
non_duplicated_rows = df[~duplicated_mask].copy()

# Step 3: Define custom logic to pick the correct median
def custom_median(group, column):
    if len(group) == 1:
        return group[column].iloc[0]  # If only one row, return it directly
    if group['Discharges'].nunique() != 1:
        idx = group['Discharges'].idxmax()  # Find the row with max Discharges
        return group.loc[idx, column]       # Return that row's median value
    else:
        return group[column].mean()          # If Discharges are all same, just take simple average

# Step 4: Merge duplicated rows
agg_dict = {
    'Discharges': 'sum',  # Sum Discharges
    'Mean Charge': lambda x: (x * duplicated_rows.loc[x.index, 'Discharges']).sum() / duplicated_rows.loc[x.index, 'Discharges'].sum(),  # Weighted average
    'Mean Cost': lambda x: (x * duplicated_rows.loc[x.index, 'Discharges']).sum() / duplicated_rows.loc[x.index, 'Discharges'].sum(),    # Weighted average
}

merged = duplicated_rows.groupby(check_cols, as_index=False).agg(agg_dict)

# Handle medians separately
medians = duplicated_rows.groupby(check_cols).apply(lambda g: pd.Series({
    'Median Charge': custom_median(g, 'Median Charge'),
    'Median Cost': custom_median(g, 'Median Cost')
})).reset_index()

# Merge the weighted averages and median results
merged_dup = pd.merge(merged, medians, on=check_cols)

# Step 5: Combine processed duplicated rows with untouched non-duplicated rows
final_df = pd.concat([merged_dup, non_duplicated_rows], ignore_index=True)

# Step 6: (Optional) Sort the final output
final_df = final_df.sort_values(check_cols).reset_index(drop=True)

# final_df is the cleaned and merged final result
final_df

<ipython-input-82-b1e199a19610>:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  medians = duplicated_rows.groupby(check_cols).apply(lambda g: pd.Series({


,Year,Facility Id,Facility Name,APR DRG Code,APR DRG Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Medical Surgical Code,APR Medical Surgical Description,Discharges,Mean Charge,Mean Cost,Median Charge,Median Cost
0,2009,1,Albany Medical Center Hospital,3,Bone Marrow Transplant,2,Moderate,P,Surgical,1.0,11279.00,4688.00,11279.00,4688.00
1,2009,1,Albany Medical Center Hospital,4,Tracheostomy W MV 96+ Hours W Extensive Proced...,2,Moderate,P,Surgical,1.0,124880.00,52943.00,124880.00,52943.00
2,2009,1,Albany Medical Center Hospital,4,Tracheostomy W MV 96+ Hours W Extensive Proced...,3,Major,P,Surgical,16.0,190065.00,68890.00,177068.00,64044.00
3,2009,1,Albany Medical Center Hospital,4,Tracheostomy W MV 96+ Hours W Extensive Proced...,4,Extreme,P,Surgical,104.0,341485.00,118244.00,291029.00,101470.00
4,2009,1,Albany Medical Center Hospital,5,Tracheostomy W MV 96+ Hours W/O Extensive Proc...,3,Major,P,Surgical,11.0,157259.00,54151.00,154174.00,50408.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192818,2021,10355,David H. Koch Center For Cancer Care,950,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,1,Minor,P,Surgical,10.0,75459.38,32054.83,80267.15,34156.67
1192819,2021,10355,David H. Koch Center For Cancer Care,950,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,2,Moderate,P,Surgical,7.0,89203.26,37249.02,99430.57,43237.60
1192820,2021,10355,David H. Koch Center For Cancer Care,950,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,3,Major,P,Surgical,1.0,144762.54,58875.36,144762.54,58875.36
1192821,2021,10355,David H. Koch Center For Cancer Care,951,MODERATELY EXTENSIVE O.R. PROCEDURE UNRELATED ...,2,Moderate,P,Surgical,2.0,17505.48,7181.56,17505.48,7181.56


In [ ]:
# Check the aggregation results of the duplicated rows
merged_dup

,Year,Facility Id,Facility Name,APR DRG Code,APR DRG Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Medical Surgical Code,APR Medical Surgical Description,Discharges,Mean Charge,Mean Cost,Median Charge,Median Cost
0,2021,1165,Jacobi Medical Center,956,UNGROUPABLE,0,Not Applicable,NA,Other,3.0,22407.256667,13299.553333,29513.620,17517.450
1,2021,1168,Montefiore Medical Center-Wakefield Hospital,956,UNGROUPABLE,0,Not Applicable,NA,Other,7.0,110208.308571,21187.891429,62031.300,15984.020
2,2021,1169,Montefiore Medical Center - Henry & Lucy Moses...,956,UNGROUPABLE,0,Not Applicable,NA,Other,8.0,208199.490000,41333.135000,203237.215,38853.285
3,2021,3058,Montefiore Med Center - Jack D Weiler Hosp of ...,956,UNGROUPABLE,0,Not Applicable,NA,Other,4.0,152909.375000,26544.477500,78737.240,16831.430


In [ ]:
# reorganize the order
new_order = ['Year', 'Facility Id', 'Facility Name', 'APR DRG Code', 'APR DRG Description',
             'APR Severity of Illness Code', 'APR Severity of Illness Description',
             'APR Medical Surgical Code', 'APR Medical Surgical Description',
             'Discharges', 'Mean Charge', 'Mean Cost', 'Median Charge', 'Median Cost']
final_df = final_df[new_order]
final_df

,Year,Facility Id,Facility Name,APR DRG Code,APR DRG Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Medical Surgical Code,APR Medical Surgical Description,Discharges,Mean Charge,Mean Cost,Median Charge,Median Cost
0,2009,1,Albany Medical Center Hospital,3,Bone Marrow Transplant,2,Moderate,P,Surgical,1.0,11279.00,4688.00,11279.00,4688.00
1,2009,1,Albany Medical Center Hospital,4,Tracheostomy W MV 96+ Hours W Extensive Proced...,2,Moderate,P,Surgical,1.0,124880.00,52943.00,124880.00,52943.00
2,2009,1,Albany Medical Center Hospital,4,Tracheostomy W MV 96+ Hours W Extensive Proced...,3,Major,P,Surgical,16.0,190065.00,68890.00,177068.00,64044.00
3,2009,1,Albany Medical Center Hospital,4,Tracheostomy W MV 96+ Hours W Extensive Proced...,4,Extreme,P,Surgical,104.0,341485.00,118244.00,291029.00,101470.00
4,2009,1,Albany Medical Center Hospital,5,Tracheostomy W MV 96+ Hours W/O Extensive Proc...,3,Major,P,Surgical,11.0,157259.00,54151.00,154174.00,50408.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192818,2021,10355,David H. Koch Center For Cancer Care,950,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,1,Minor,P,Surgical,10.0,75459.38,32054.83,80267.15,34156.67
1192819,2021,10355,David H. Koch Center For Cancer Care,950,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,2,Moderate,P,Surgical,7.0,89203.26,37249.02,99430.57,43237.60
1192820,2021,10355,David H. Koch Center For Cancer Care,950,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,3,Major,P,Surgical,1.0,144762.54,58875.36,144762.54,58875.36
1192821,2021,10355,David H. Koch Center For Cancer Care,951,MODERATELY EXTENSIVE O.R. PROCEDURE UNRELATED ...,2,Moderate,P,Surgical,2.0,17505.48,7181.56,17505.48,7181.56


### 1.4 Add neccessary columns

In [ ]:
# Create columns 'Mean Profit', 'Total Charges', 'Total Costs'

final_df['Mean Profit'] = final_df['Mean Charge'] - final_df['Mean Cost']
final_df['Total Charges'] = final_df['Discharges'] * final_df['Mean Charge']
final_df['Total Costs'] = final_df['Discharges'] * final_df['Mean Cost']
final_df

,Year,Facility Id,Facility Name,APR DRG Code,APR DRG Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Medical Surgical Code,APR Medical Surgical Description,Discharges,Mean Charge,Mean Cost,Median Charge,Median Cost,Mean Profit,Total Charges,Total Costs
0,2009,1,Albany Medical Center Hospital,3,Bone Marrow Transplant,2,Moderate,P,Surgical,1.0,11279.00,4688.00,11279.00,4688.00,6591.00,11279.00,4688.00
1,2009,1,Albany Medical Center Hospital,4,Tracheostomy W MV 96+ Hours W Extensive Proced...,2,Moderate,P,Surgical,1.0,124880.00,52943.00,124880.00,52943.00,71937.00,124880.00,52943.00
2,2009,1,Albany Medical Center Hospital,4,Tracheostomy W MV 96+ Hours W Extensive Proced...,3,Major,P,Surgical,16.0,190065.00,68890.00,177068.00,64044.00,121175.00,3041040.00,1102240.00
3,2009,1,Albany Medical Center Hospital,4,Tracheostomy W MV 96+ Hours W Extensive Proced...,4,Extreme,P,Surgical,104.0,341485.00,118244.00,291029.00,101470.00,223241.00,35514440.00,12297376.00
4,2009,1,Albany Medical Center Hospital,5,Tracheostomy W MV 96+ Hours W/O Extensive Proc...,3,Major,P,Surgical,11.0,157259.00,54151.00,154174.00,50408.00,103108.00,1729849.00,595661.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192818,2021,10355,David H. Koch Center For Cancer Care,950,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,1,Minor,P,Surgical,10.0,75459.38,32054.83,80267.15,34156.67,43404.55,754593.80,320548.30
1192819,2021,10355,David H. Koch Center For Cancer Care,950,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,2,Moderate,P,Surgical,7.0,89203.26,37249.02,99430.57,43237.60,51954.24,624422.82,260743.14
1192820,2021,10355,David H. Koch Center For Cancer Care,950,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,3,Major,P,Surgical,1.0,144762.54,58875.36,144762.54,58875.36,85887.18,144762.54,58875.36
1192821,2021,10355,David H. Koch Center For Cancer Care,951,MODERATELY EXTENSIVE O.R. PROCEDURE UNRELATED ...,2,Moderate,P,Surgical,2.0,17505.48,7181.56,17505.48,7181.56,10323.92,35010.96,14363.12


### Identigy Latitude and Longtiude by Facility Name

In [ ]:
# get list of Facility Name
unique_facilities = df[['Facility Name']]
unique_facilities = unique_facilities.drop_duplicates()

In [ ]:
import time
from geopy.geocoders import Nominatim

# Initialize geolocator with a user agent
geolocator = Nominatim(user_agent="myGeocoderApp")

# Function to get latitude and longitude with retry logic
def get_lat_long(place_name, retries=3, timeout=5, delay=2):
    for attempt in range(retries):
        try:
            location = geolocator.geocode(place_name, timeout=timeout)  # Increase timeout
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except Exception as e:
            print(f"Error with {place_name}: {e}")
            return None, None
    return None, None

# Function to apply geocoding and store lat/long using .loc
def process_geocode(df):
    for idx, row in df.iterrows():  # iterating over the rows
        place_name = row['Facility Name']
        lat, lon = get_lat_long(place_name)

        # Use .loc to set the values in the correct place
        df.loc[idx, 'place_latitude'] = lat
        df.loc[idx, 'place_longitude'] = lon

        # Optional: Adding a delay to avoid rate-limiting
        time.sleep(1)

    return df

# Process the geocoding for the entire DataFrame
unique_facilities = process_geocode(unique_facilities)
unique_facilities.to_csv('Facility location.csv', index=False)

add Null Facility manually in Excel, then import file to merge with the dataset.

In [ ]:
# Add the facility location latitude and lontitude
# Import the facility location data
facility_location = pd.read_csv('Facility location.csv')

facility_location[facility_location['Facility Name'] == 'New York Hospital Medical Center of Queens']


,Facility Name,place_latitude,place_longitude
163,New York Hospital Medical Center of Queens,40.747257,-73.825187


### Merge latitude and longtitude to dataset

In [ ]:
# Left join the data with location
# Clean facility names: strip spaces, lower case (optional)
final_df['Facility Name'] = final_df['Facility Name'].str.strip()
facility_location['Facility Name'] = facility_location['Facility Name'].str.strip()


final_df = pd.merge(final_df, facility_location, on='Facility Name', how='left')
final_df

,Year,Facility Id,Facility Name,APR DRG Code,APR DRG Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Medical Surgical Code,APR Medical Surgical Description,Discharges,Mean Charge,Mean Cost,Median Charge,Median Cost,Mean Profit,Total Charges,Total Costs,place_latitude,place_longitude
0,2009,1,Albany Medical Center Hospital,3,Bone Marrow Transplant,2,Moderate,P,Surgical,1.0,11279.00,4688.00,11279.00,4688.00,6591.00,11279.00,4688.00,42.653280,-73.777179
1,2009,1,Albany Medical Center Hospital,4,Tracheostomy W MV 96+ Hours W Extensive Proced...,2,Moderate,P,Surgical,1.0,124880.00,52943.00,124880.00,52943.00,71937.00,124880.00,52943.00,42.653280,-73.777179
2,2009,1,Albany Medical Center Hospital,4,Tracheostomy W MV 96+ Hours W Extensive Proced...,3,Major,P,Surgical,16.0,190065.00,68890.00,177068.00,64044.00,121175.00,3041040.00,1102240.00,42.653280,-73.777179
3,2009,1,Albany Medical Center Hospital,4,Tracheostomy W MV 96+ Hours W Extensive Proced...,4,Extreme,P,Surgical,104.0,341485.00,118244.00,291029.00,101470.00,223241.00,35514440.00,12297376.00,42.653280,-73.777179
4,2009,1,Albany Medical Center Hospital,5,Tracheostomy W MV 96+ Hours W/O Extensive Proc...,3,Major,P,Surgical,11.0,157259.00,54151.00,154174.00,50408.00,103108.00,1729849.00,595661.00,42.653280,-73.777179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192818,2021,10355,David H. Koch Center For Cancer Care,950,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,1,Minor,P,Surgical,10.0,75459.38,32054.83,80267.15,34156.67,43404.55,754593.80,320548.30,40.767365,-73.950412
1192819,2021,10355,David H. Koch Center For Cancer Care,950,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,2,Moderate,P,Surgical,7.0,89203.26,37249.02,99430.57,43237.60,51954.24,624422.82,260743.14,40.767365,-73.950412
1192820,2021,10355,David H. Koch Center For Cancer Care,950,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,3,Major,P,Surgical,1.0,144762.54,58875.36,144762.54,58875.36,85887.18,144762.54,58875.36,40.767365,-73.950412
1192821,2021,10355,David H. Koch Center For Cancer Care,951,MODERATELY EXTENSIVE O.R. PROCEDURE UNRELATED ...,2,Moderate,P,Surgical,2.0,17505.48,7181.56,17505.48,7181.56,10323.92,35010.96,14363.12,40.767365,-73.950412


In [ ]:
final_df['place_latitude'].fillna(40.7472567, inplace=True)
final_df['place_longitude'].fillna(-73.8251866, inplace=True)
final_df.isna().sum()

<ipython-input-88-c545c6e132f9>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['place_latitude'].fillna(40.7472567, inplace=True)
<ipython-input-88-c545c6e132f9>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)

,0
Year,0
Facility Id,0
Facility Name,0
APR DRG Code,0
APR DRG Description,0
APR Severity of Illness Code,0
APR Severity of Illness Description,0
APR Medical Surgical Code,0
APR Medical Surgical Description,0
Discharges,0


In [ ]:
final_df.loc[
    ((final_df['place_latitude'] < 40) | (final_df['place_latitude'] > 45)) |
    ((final_df['place_longitude'] < -80) | (final_df['place_longitude'] > -72))
].groupby('Facility Name')[['place_latitude', 'place_longitude']].agg('first')

,place_latitude,place_longitude
Facility Name,,


In [ ]:
#final_df.to_csv('data.csv', index=False)

In [ ]:
final_df.groupby('Year').size()

,0
Year,
2009,129255
2010,128014
2011,126288
2012,123600
2013,117226
2014,114376
2015,114581
2016,114602
2017,113730
